In [ ]:
!pip install numpy pandas scikit-learn tensorflow opencv-python opencv-contrib-python


In [1]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import tensorflow as tf

2023-05-29 22:19:27.977652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 22:19:28.188020: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-29 22:19:28.220429: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 22:19:28.220445: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:

# Load the annotations.csv file
annotations = pd.read_csv('annotations.csv')

# Extract the required input features and output
X = annotations[['xmin', 'ymin', 'xmax', 'ymax', 'xloc', 'yloc', 'observation angle']]
y = annotations['zloc']

# Normalize the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [3]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:

# Create the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(150, activation='relu', input_shape=(7,)),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Create the training_epochs folder if it doesn't exist
if not os.path.exists('training_epochs'):
    os.makedirs('training_epochs')

# Create a ModelCheckpoint callback to save the model after every 100 epochs
checkpoint_callback = ModelCheckpoint(
    filepath='training_epochs/trained_model_epoch_{epoch:04d}.h5',
    save_freq=100 * len(X_train) // 32,  # Save after every 100 epochs
    save_weights_only=False,
    save_best_only=False
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=3000,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=2,
    callbacks=[checkpoint_callback]
)


In [21]:
# Create the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(150, activation='relu', input_shape=(7,)),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Create the training_epochs folder if it doesn't exist
if not os.path.exists('training_epochs'):
    os.makedirs('training_epochs')

epochs_list = [100]
validation_losses = []

for epochs in epochs_list:
    # Create a ModelCheckpoint callback to save the model after every specified number of epochs
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'training_epochs/trained_model_{epochs:04d}.h5',
        save_freq=100,
        save_weights_only=False,
        save_best_only=False
    )

    # Train the model for the specified number of epochs
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=2,
        callbacks=[checkpoint_callback]
    )

    # Plot the validation loss history
    plt.plot(history.history['val_loss'], label=f'{epochs} Epochs')
    validation_losses.append(history.history['val_loss'][-1])

    # Save the validation loss graph
    plt.xlabel('Epochs')
    plt.ylabel('Validation Loss')
    plt.legend()
    plt.savefig(f'metrics/validation_loss_{epochs:04d}.png')
    plt.close()



Epoch 1/100
1015/1015 - 5s - loss: 50.2526 - mae: 3.6301 - val_loss: 8.3486 - val_mae: 1.8181 - 5s/epoch - 5ms/step
Epoch 2/100
1015/1015 - 1s - loss: 5.6351 - mae: 1.4127 - val_loss: 3.4755 - val_mae: 1.0957 - 1s/epoch - 1ms/step
Epoch 3/100
1015/1015 - 1s - loss: 3.8654 - mae: 1.1779 - val_loss: 2.6745 - val_mae: 1.0111 - 1s/epoch - 1ms/step
Epoch 4/100
1015/1015 - 1s - loss: 3.1149 - mae: 1.0781 - val_loss: 2.2064 - val_mae: 0.9376 - 1s/epoch - 1ms/step
Epoch 5/100
1015/1015 - 1s - loss: 2.4608 - mae: 0.9697 - val_loss: 1.8760 - val_mae: 0.8777 - 1s/epoch - 1ms/step
Epoch 6/100
1015/1015 - 1s - loss: 2.2021 - mae: 0.9226 - val_loss: 1.9952 - val_mae: 0.9754 - 1s/epoch - 1ms/step
Epoch 7/100
1015/1015 - 1s - loss: 1.9124 - mae: 0.8651 - val_loss: 1.5100 - val_mae: 0.8081 - 1s/epoch - 1ms/step
Epoch 8/100
1015/1015 - 1s - loss: 1.7227 - mae: 0.8325 - val_loss: 1.1489 - val_mae: 0.7397 - 1s/epoch - 1ms/step
Epoch 9/100
1015/1015 - 1s - loss: 1.5891 - mae: 0.7892 - val_loss: 1.4509 - va

Epoch 70/100
1015/1015 - 1s - loss: 0.4325 - mae: 0.4165 - val_loss: 0.5514 - val_mae: 0.4886 - 921ms/epoch - 908us/step
Epoch 71/100
1015/1015 - 1s - loss: 0.3964 - mae: 0.3946 - val_loss: 0.7342 - val_mae: 0.4475 - 938ms/epoch - 924us/step
Epoch 72/100
1015/1015 - 1s - loss: 0.4849 - mae: 0.4291 - val_loss: 0.3772 - val_mae: 0.3835 - 928ms/epoch - 914us/step
Epoch 73/100
1015/1015 - 1s - loss: 0.4296 - mae: 0.3985 - val_loss: 0.3229 - val_mae: 0.3510 - 905ms/epoch - 891us/step
Epoch 74/100
1015/1015 - 1s - loss: 0.4369 - mae: 0.4081 - val_loss: 0.3880 - val_mae: 0.3799 - 932ms/epoch - 918us/step
Epoch 75/100
1015/1015 - 1s - loss: 0.3209 - mae: 0.3555 - val_loss: 0.3326 - val_mae: 0.3842 - 920ms/epoch - 906us/step
Epoch 76/100
1015/1015 - 1s - loss: 0.4297 - mae: 0.4056 - val_loss: 0.3375 - val_mae: 0.3732 - 901ms/epoch - 887us/step
Epoch 77/100
1015/1015 - 1s - loss: 0.4549 - mae: 0.4089 - val_loss: 0.2886 - val_mae: 0.3408 - 893ms/epoch - 880us/step
Epoch 78/100
1015/1015 - 1s - lo

In [6]:
import numpy as np

# Create the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(150, activation='relu', input_shape=(7,)),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Create the training_epochs folder if it doesn't exist
if not os.path.exists('training_epochs'):
    os.makedirs('training_epochs')

epochs_list = [3000]
validation_losses = []

for epochs in epochs_list:
    # Create a ModelCheckpoint callback to save the model after every specified number of epochs
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'training_epochs/trained_model_{epochs:04d}.h5',
        save_freq=100,
        save_weights_only=False,
        save_best_only=False
    )

    # Train the model for the specified number of epochs
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=2,
        callbacks=[checkpoint_callback]
    )

    # Get the best validation loss value
    best_val_loss = np.min(history.history['val_loss'])

    # Store only the best values for training and validation loss
    filtered_train_loss = [history.history['loss'][0]]
    filtered_val_loss = [history.history['val_loss'][0]]

    for val_loss in history.history['val_loss']:
        if val_loss < filtered_val_loss[-1]:
            filtered_val_loss.append(val_loss)

    for train_loss in history.history['loss']:
        if train_loss < filtered_train_loss[-1]:
            filtered_train_loss.append(train_loss)

    # Plot the training loss history
    plt.plot(filtered_train_loss, label=f'{epochs} Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Training Loss')
    plt.legend()
    plt.savefig(f'metrics/training_loss_{epochs:04d}.png')
    plt.close()

    # Plot the validation loss history
    plt.plot(filtered_val_loss, label=f'{epochs} Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Validation Loss')
    plt.legend()
    plt.savefig(f'metrics/validation_loss_{epochs:04d}.png')
    plt.close()

    # Store the best validation loss
    validation_losses.append(best_val_loss)

# Plot the combined graph of training and validation loss
plt.plot(filtered_train_loss, label='Training Loss')
plt.plot(filtered_val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('metrics/training_validation_loss.png')
plt.close()


Epoch 1/3000
1015/1015 - 1s - loss: 48.0580 - mae: 3.6830 - val_loss: 8.4996 - val_mae: 1.8638 - 1s/epoch - 1ms/step
Epoch 2/3000
1015/1015 - 1s - loss: 6.1112 - mae: 1.4482 - val_loss: 5.4926 - val_mae: 1.6707 - 928ms/epoch - 915us/step
Epoch 3/3000
1015/1015 - 1s - loss: 4.1604 - mae: 1.2068 - val_loss: 2.8224 - val_mae: 0.9831 - 873ms/epoch - 861us/step
Epoch 4/3000
1015/1015 - 1s - loss: 3.2068 - mae: 1.0641 - val_loss: 2.4112 - val_mae: 1.0118 - 913ms/epoch - 899us/step
Epoch 5/3000
1015/1015 - 1s - loss: 2.7069 - mae: 0.9834 - val_loss: 2.5056 - val_mae: 1.0378 - 863ms/epoch - 850us/step
Epoch 6/3000
1015/1015 - 1s - loss: 2.2451 - mae: 0.9104 - val_loss: 2.2573 - val_mae: 0.9450 - 906ms/epoch - 892us/step
Epoch 7/3000
1015/1015 - 1s - loss: 2.0254 - mae: 0.8642 - val_loss: 1.1604 - val_mae: 0.6812 - 910ms/epoch - 896us/step
Epoch 8/3000
1015/1015 - 1s - loss: 1.8741 - mae: 0.8530 - val_loss: 1.6751 - val_mae: 0.8749 - 882ms/epoch - 869us/step
Epoch 9/3000
1015/1015 - 1s - loss: 

Epoch 70/3000
1015/1015 - 1s - loss: 0.4757 - mae: 0.4210 - val_loss: 0.4174 - val_mae: 0.4534 - 972ms/epoch - 958us/step
Epoch 71/3000
1015/1015 - 1s - loss: 0.3592 - mae: 0.3774 - val_loss: 0.4924 - val_mae: 0.4602 - 993ms/epoch - 978us/step
Epoch 72/3000
1015/1015 - 1s - loss: 0.3685 - mae: 0.3877 - val_loss: 0.6376 - val_mae: 0.5733 - 974ms/epoch - 960us/step
Epoch 73/3000
1015/1015 - 1s - loss: 0.3559 - mae: 0.3859 - val_loss: 0.3779 - val_mae: 0.4267 - 1s/epoch - 987us/step
Epoch 74/3000
1015/1015 - 1s - loss: 0.3763 - mae: 0.3858 - val_loss: 0.4343 - val_mae: 0.3874 - 1s/epoch - 1ms/step
Epoch 75/3000
1015/1015 - 1s - loss: 0.4703 - mae: 0.4135 - val_loss: 0.3211 - val_mae: 0.3801 - 1s/epoch - 992us/step
Epoch 76/3000
1015/1015 - 1s - loss: 0.3747 - mae: 0.3818 - val_loss: 0.2943 - val_mae: 0.3410 - 1s/epoch - 1ms/step
Epoch 77/3000
1015/1015 - 1s - loss: 0.3292 - mae: 0.3722 - val_loss: 0.3515 - val_mae: 0.3688 - 1s/epoch - 1ms/step
Epoch 78/3000
1015/1015 - 1s - loss: 0.3663 -

1015/1015 - 1s - loss: 0.2272 - mae: 0.3117 - val_loss: 0.2893 - val_mae: 0.3375 - 1s/epoch - 988us/step
Epoch 139/3000
1015/1015 - 1s - loss: 0.2381 - mae: 0.3146 - val_loss: 0.3006 - val_mae: 0.3675 - 993ms/epoch - 979us/step
Epoch 140/3000
1015/1015 - 1s - loss: 0.2141 - mae: 0.3046 - val_loss: 0.4045 - val_mae: 0.3688 - 979ms/epoch - 965us/step
Epoch 141/3000
1015/1015 - 1s - loss: 0.2531 - mae: 0.3258 - val_loss: 0.2737 - val_mae: 0.3429 - 1s/epoch - 990us/step
Epoch 142/3000
1015/1015 - 1s - loss: 0.2042 - mae: 0.2976 - val_loss: 0.1989 - val_mae: 0.2821 - 979ms/epoch - 964us/step
Epoch 143/3000
1015/1015 - 1s - loss: 0.2251 - mae: 0.3073 - val_loss: 0.2496 - val_mae: 0.3072 - 986ms/epoch - 971us/step
Epoch 144/3000
1015/1015 - 1s - loss: 0.2386 - mae: 0.3243 - val_loss: 0.3658 - val_mae: 0.3761 - 967ms/epoch - 953us/step
Epoch 145/3000
1015/1015 - 1s - loss: 0.2615 - mae: 0.3270 - val_loss: 0.3890 - val_mae: 0.4178 - 976ms/epoch - 962us/step
Epoch 146/3000
1015/1015 - 1s - loss:

Epoch 206/3000
1015/1015 - 1s - loss: 0.1676 - mae: 0.2673 - val_loss: 0.1746 - val_mae: 0.2457 - 1s/epoch - 1ms/step
Epoch 207/3000
1015/1015 - 1s - loss: 0.1975 - mae: 0.2859 - val_loss: 0.1820 - val_mae: 0.2522 - 1s/epoch - 1ms/step
Epoch 208/3000
1015/1015 - 1s - loss: 0.1443 - mae: 0.2533 - val_loss: 0.2472 - val_mae: 0.3463 - 1s/epoch - 1ms/step
Epoch 209/3000
1015/1015 - 1s - loss: 0.1777 - mae: 0.2754 - val_loss: 0.3774 - val_mae: 0.3769 - 1s/epoch - 1ms/step
Epoch 210/3000
1015/1015 - 1s - loss: 0.1669 - mae: 0.2699 - val_loss: 0.1983 - val_mae: 0.2680 - 1s/epoch - 1ms/step
Epoch 211/3000
1015/1015 - 1s - loss: 0.1704 - mae: 0.2696 - val_loss: 0.5584 - val_mae: 0.4619 - 1s/epoch - 1ms/step
Epoch 212/3000
1015/1015 - 1s - loss: 0.2325 - mae: 0.2881 - val_loss: 0.2011 - val_mae: 0.3107 - 1s/epoch - 1ms/step
Epoch 213/3000
1015/1015 - 1s - loss: 0.1376 - mae: 0.2499 - val_loss: 0.2582 - val_mae: 0.3282 - 1s/epoch - 1ms/step
Epoch 214/3000
1015/1015 - 1s - loss: 0.2019 - mae: 0.28

Epoch 276/3000
1015/1015 - 1s - loss: 0.1186 - mae: 0.2321 - val_loss: 0.1634 - val_mae: 0.2494 - 1s/epoch - 1ms/step
Epoch 277/3000
1015/1015 - 1s - loss: 0.1412 - mae: 0.2503 - val_loss: 0.1698 - val_mae: 0.2656 - 1s/epoch - 1ms/step
Epoch 278/3000
1015/1015 - 1s - loss: 0.1435 - mae: 0.2508 - val_loss: 0.1887 - val_mae: 0.2628 - 1s/epoch - 995us/step
Epoch 279/3000
1015/1015 - 1s - loss: 0.1855 - mae: 0.2777 - val_loss: 0.2068 - val_mae: 0.2943 - 997ms/epoch - 982us/step
Epoch 280/3000
1015/1015 - 1s - loss: 0.1290 - mae: 0.2373 - val_loss: 0.1486 - val_mae: 0.2239 - 1s/epoch - 997us/step
Epoch 281/3000
1015/1015 - 1s - loss: 0.1308 - mae: 0.2407 - val_loss: 0.1398 - val_mae: 0.2290 - 1s/epoch - 993us/step
Epoch 282/3000
1015/1015 - 1s - loss: 0.1169 - mae: 0.2307 - val_loss: 0.1680 - val_mae: 0.2511 - 1s/epoch - 1ms/step
Epoch 283/3000
1015/1015 - 1s - loss: 0.1409 - mae: 0.2485 - val_loss: 0.2081 - val_mae: 0.2988 - 1s/epoch - 988us/step
Epoch 284/3000
1015/1015 - 1s - loss: 0.156

...

Epoch 2698/3000
1015/1015 - 1s - loss: 0.0407 - mae: 0.1411 - val_loss: 0.1303 - val_mae: 0.2156 - 1s/epoch - 1ms/step
Epoch 2699/3000
1015/1015 - 1s - loss: 0.0395 - mae: 0.1401 - val_loss: 0.1044 - val_mae: 0.1786 - 1s/epoch - 1ms/step
Epoch 2700/3000
1015/1015 - 1s - loss: 0.0356 - mae: 0.1355 - val_loss: 0.1205 - val_mae: 0.1988 - 1s/epoch - 1ms/step
Epoch 2701/3000
1015/1015 - 1s - loss: 0.0413 - mae: 0.1415 - val_loss: 0.1669 - val_mae: 0.2708 - 1s/epoch - 1ms/step
Epoch 2702/3000
1015/1015 - 1s - loss: 0.0381 - mae: 0.1378 - val_loss: 0.1020 - val_mae: 0.1707 - 1s/epoch - 1ms/step
Epoch 2703/3000
1015/1015 - 1s - loss: 0.0338 - mae: 0.1309 - val_loss: 0.1252 - val_mae: 0.2084 - 1s/epoch - 1ms/step
Epoch 2704/3000
1015/1015 - 1s - loss: 0.0428 - mae: 0.1418 - val_loss: 0.1060 - val_mae: 0.1853 - 1s/epoch - 1ms/step
Epoch 2705/3000
1015/1015 - 1s - loss: 0.0341 - mae: 0.1324 - val_loss: 0.1082 - val_mae: 0.1820 - 1s/epoch - 1ms/step
Epoch 2706/3000
1015/1015 - 1s - loss: 0.0385 - 

Epoch 2767/3000
1015/1015 - 1s - loss: 0.0480 - mae: 0.1468 - val_loss: 0.1183 - val_mae: 0.2073 - 1s/epoch - 1ms/step
Epoch 2768/3000
1015/1015 - 1s - loss: 0.0322 - mae: 0.1275 - val_loss: 0.1050 - val_mae: 0.1843 - 1s/epoch - 1ms/step
Epoch 2769/3000
1015/1015 - 1s - loss: 0.0379 - mae: 0.1380 - val_loss: 0.1393 - val_mae: 0.2220 - 1s/epoch - 1ms/step
Epoch 2770/3000
1015/1015 - 1s - loss: 0.0363 - mae: 0.1354 - val_loss: 0.1032 - val_mae: 0.1811 - 1s/epoch - 1ms/step
Epoch 2771/3000
1015/1015 - 1s - loss: 0.0329 - mae: 0.1304 - val_loss: 0.1115 - val_mae: 0.1957 - 1s/epoch - 1ms/step
Epoch 2772/3000
1015/1015 - 1s - loss: 0.0437 - mae: 0.1456 - val_loss: 0.1114 - val_mae: 0.1897 - 1s/epoch - 1ms/step
Epoch 2773/3000
1015/1015 - 1s - loss: 0.0352 - mae: 0.1321 - val_loss: 0.0975 - val_mae: 0.1731 - 1s/epoch - 1ms/step
Epoch 2774/3000
1015/1015 - 1s - loss: 0.0343 - mae: 0.1328 - val_loss: 0.1081 - val_mae: 0.1745 - 1s/epoch - 1ms/step
Epoch 2775/3000
1015/1015 - 1s - loss: 0.0372 - 

Epoch 2836/3000
1015/1015 - 1s - loss: 0.0357 - mae: 0.1340 - val_loss: 0.0975 - val_mae: 0.1678 - 1s/epoch - 1ms/step
Epoch 2837/3000
1015/1015 - 1s - loss: 0.0382 - mae: 0.1375 - val_loss: 0.0975 - val_mae: 0.1659 - 1s/epoch - 1ms/step
Epoch 2838/3000
1015/1015 - 1s - loss: 0.0320 - mae: 0.1282 - val_loss: 0.1042 - val_mae: 0.1726 - 1s/epoch - 1ms/step
Epoch 2839/3000
1015/1015 - 1s - loss: 0.0377 - mae: 0.1368 - val_loss: 0.1058 - val_mae: 0.1719 - 1s/epoch - 1ms/step
Epoch 2840/3000
1015/1015 - 1s - loss: 0.0361 - mae: 0.1356 - val_loss: 0.1051 - val_mae: 0.1817 - 1s/epoch - 1ms/step
Epoch 2841/3000
1015/1015 - 1s - loss: 0.0335 - mae: 0.1313 - val_loss: 0.1081 - val_mae: 0.1867 - 1s/epoch - 1ms/step
Epoch 2842/3000
1015/1015 - 1s - loss: 0.0401 - mae: 0.1415 - val_loss: 0.1716 - val_mae: 0.2893 - 1s/epoch - 1ms/step
Epoch 2843/3000
1015/1015 - 1s - loss: 0.0376 - mae: 0.1368 - val_loss: 0.1012 - val_mae: 0.1767 - 1s/epoch - 1ms/step
Epoch 2844/3000
1015/1015 - 1s - loss: 0.0410 - 

Epoch 2905/3000
1015/1015 - 1s - loss: 0.0346 - mae: 0.1329 - val_loss: 0.1169 - val_mae: 0.1936 - 1s/epoch - 1ms/step
Epoch 2906/3000
1015/1015 - 1s - loss: 0.0444 - mae: 0.1384 - val_loss: 0.0973 - val_mae: 0.1752 - 1s/epoch - 1ms/step
Epoch 2907/3000
1015/1015 - 1s - loss: 0.0314 - mae: 0.1263 - val_loss: 0.0956 - val_mae: 0.1687 - 1s/epoch - 1ms/step
Epoch 2908/3000
1015/1015 - 1s - loss: 0.0364 - mae: 0.1362 - val_loss: 0.1035 - val_mae: 0.1821 - 1s/epoch - 1ms/step
Epoch 2909/3000
1015/1015 - 1s - loss: 0.0346 - mae: 0.1320 - val_loss: 0.1283 - val_mae: 0.2086 - 1s/epoch - 1ms/step
Epoch 2910/3000
1015/1015 - 1s - loss: 0.0416 - mae: 0.1404 - val_loss: 0.1169 - val_mae: 0.2014 - 1s/epoch - 1ms/step
Epoch 2911/3000
1015/1015 - 1s - loss: 0.0327 - mae: 0.1288 - val_loss: 0.0978 - val_mae: 0.1767 - 1s/epoch - 1ms/step
Epoch 2912/3000
1015/1015 - 1s - loss: 0.0341 - mae: 0.1321 - val_loss: 0.1106 - val_mae: 0.1979 - 1s/epoch - 1ms/step
Epoch 2913/3000
1015/1015 - 1s - loss: 0.0399 - 

Epoch 2974/3000
1015/1015 - 1s - loss: 0.0382 - mae: 0.1361 - val_loss: 0.1111 - val_mae: 0.1994 - 1s/epoch - 1ms/step
Epoch 2975/3000
1015/1015 - 1s - loss: 0.0312 - mae: 0.1269 - val_loss: 0.0966 - val_mae: 0.1696 - 1s/epoch - 1ms/step
Epoch 2976/3000
1015/1015 - 1s - loss: 0.0365 - mae: 0.1355 - val_loss: 0.1047 - val_mae: 0.1911 - 1s/epoch - 1ms/step
Epoch 2977/3000
1015/1015 - 1s - loss: 0.0358 - mae: 0.1324 - val_loss: 0.1142 - val_mae: 0.1907 - 1s/epoch - 1ms/step
Epoch 2978/3000
1015/1015 - 1s - loss: 0.0403 - mae: 0.1390 - val_loss: 0.1054 - val_mae: 0.1762 - 1s/epoch - 1ms/step
Epoch 2979/3000
1015/1015 - 1s - loss: 0.0323 - mae: 0.1278 - val_loss: 0.1121 - val_mae: 0.1778 - 1s/epoch - 1ms/step
Epoch 2980/3000
1015/1015 - 1s - loss: 0.0361 - mae: 0.1321 - val_loss: 0.0941 - val_mae: 0.1702 - 1s/epoch - 1ms/step
Epoch 2981/3000
1015/1015 - 1s - loss: 0.0352 - mae: 0.1343 - val_loss: 0.0971 - val_mae: 0.1689 - 1s/epoch - 1ms/step
Epoch 2982/3000
1015/1015 - 1s - loss: 0.0356 - 

In [ ]:
import matplotlib.pyplot as plt

# Define the epochs for which you want to plot the MAE
epochs = [100, 500, 1000, 2000]

# Create empty lists to store the MAE values for each epoch
mae_values = []

# Iterate over the epochs
for epoch in epochs:
    # Load the trained model for the specific epoch
    model.load_weights(f'training_epochs/trained_model_{epoch}.h5')
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate the MAE
    mae = mean_absolute_error(y_test, y_pred)
    
    # Append the MAE value to the list
    mae_values.append(mae)

# Plot the MAE values
plt.plot(epochs, mae_values, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('MAE for Different Epochs')
plt.grid(True)
plt.show()

In [ ]:

# Train the model with the final number of epochs
"""final_epochs = 2000
final_history = model.fit(
    X_train, y_train,
    epochs=final_epochs,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=2
)"""
model = tf.keras.models.load_model('training_epochs/trained_model_2000.h5')

# Generate performance metrics for the final model
y_pred = model.predict(X_test)
abs_error = mean_absolute_error(y_test, y_pred)
rel_error = mean_absolute_percentage_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Create a table with performance metrics
metrics_table = pd.DataFrame({
    'Metric': ['Absolute Error', 'Relative Error', 'Mean Absolute Percentage Error', 'Mean Absolute Error', 'R2 Score'],
    'Value': [abs_error, rel_error, mape, mae, r2]
})

# Save the metrics table as a CSV file
metrics_table.to_csv('metrics/metrics_table.csv', index=False)

# Save the metrics table as a PNG image
plt.figure(figsize=(6, 4))
plt.axis('off')
plt.table(cellText=metrics_table.values,
          colLabels=metrics_table.columns,
          cellLoc='center',
          loc='center')
plt.savefig('metrics/metrics_table.png')
plt.close()

# Plot the graph showing the metrics relative to the number of epochs
metrics = {
    'Absolute Error': [],
    'Relative Error': [],
    'Mean Absolute Percentage Error': [],
    'Mean Absolute Error': [],
    'R2 Score': []
}

for i, epochs in enumerate(epochs_list):
    metrics['Absolute Error'].append(mean_absolute_error(y_test, model.predict(X_test, batch_size=32)))
    metrics['Relative Error'].append(mean_absolute_percentage_error(y_test, model.predict(X_test, batch_size=32)))
    metrics['Mean Absolute Percentage Error'].append(mean_absolute_percentage_error(y_test, model.predict(X_test, batch_size=32)))
    metrics['Mean Absolute Error'].append(mean_absolute_error(y_test, model.predict(X_test, batch_size=32)))
    metrics['R2 Score'].append(r2_score(y_test, model.predict(X_test, batch_size=32)))

metrics['Absolute Error'].append(abs_error)
metrics['Relative Error'].append(rel_error)
metrics['Mean Absolute Percentage Error'].append(mape)
metrics['Mean Absolute Error'].append(mae)
metrics['R2 Score'].append(r2)

plt.figure(figsize=(10, 6))
for metric, values in metrics.items():
    plt.plot(epochs_list[:i+1] + [final_epochs], values[:i+1] + [values[-1]], marker='o')
plt.xlabel('Number of Epochs')
plt.ylabel('Metrics')
plt.xticks(epochs_list + [final_epochs])
plt.legend(metrics.keys())
plt.savefig('metrics/metrics_graph.png')
plt.close()

In [ ]:
#model.save('trained_model.h5')


In [ ]:

# Plot the training history
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.show()


In [ ]:
!pip install yolov3-tf2


In [5]:

model = tf.keras.models.load_model('trained_model.h5')


2023-05-24 11:48:28.098858: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-24 11:48:28.099213: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sai/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-24 11:48:28.099304: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sai/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-24 11:48:28.099381: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dyn

In [6]:


def detect_objects(image, model, scaler):
    # Load the YOLOv3 model
    net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

    # Get the output layer names
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


    # Prepare the input image
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Run the forward pass
    layer_outputs = net.forward(output_layers)

    # Extract the detected objects and their bounding boxes, camera coordinates, and observation angles
    objects = []
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Get the bounding box coordinates
                center_x, center_y, width, height = (detection[0:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])).astype('int')
                xmin, ymin = int(center_x - width / 2), int(center_y - height / 2)
                xmax, ymax = int(center_x + width / 2), int(center_y + height / 2)

                # Get the camera coordinates and observation angle
                xloc, yloc = center_x / image.shape[1], center_y / image.shape[0]
                observation_angle = np.arctan2(yloc, xloc)

                # Prepare the input for the multilayer perceptron model
                input_data = np.array([[xmin, ymin, xmax, ymax, xloc, yloc, observation_angle]])
                input_data_scaled = scaler.transform(input_data)

                # Predict the distance using the trained model
                zloc = model.predict(input_data_scaled)[0][0]

                # Store the detected object information
                objects.append({
                    'class_id': class_id,
                    'confidence': confidence,
                    'bounding_box': (xmin, ymin, xmax, ymax),
                    'camera_coordinates': (xloc, yloc),
                    'observation_angle': observation_angle,
                    'zloc': zloc
                })

    return objects


In [7]:
def process_files(input_path, output_path, mode):
    # Load the class names
    with open('yolov3.names', 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    # Get the list of files in the input directory
    input_files = [os.path.join(input_path, f) for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]

    # Define the confidence and NMS thresholds
    CONF_THRESHOLD = 0.1  # Increase this value to reduce false positives
    NMS_THRESHOLD = 0.1   # Increase this value to allow more overlap between bounding boxes
    
    for input_file in input_files:
        # Read the input image or video
        if mode == 'image':
            image = cv2.imread(input_file)
            # Detect objects and draw the bounding boxes, class names, and distances
            objects = detect_objects(image, model, scaler)
            indices = cv2.dnn.NMSBoxes([obj['bounding_box'] for obj in objects], [obj['confidence'] for obj in objects], CONF_THRESHOLD, NMS_THRESHOLD)

            for index in indices.flatten():
                obj = objects[index]
                xmin, ymin, xmax, ymax = obj['bounding_box']
                class_name = classes[obj['class_id']]
                distance = obj['zloc']
                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(image, f'{class_name} {distance:.2f}m', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.imwrite(os.path.join(output_path, os.path.basename(input_file)), image)
        elif mode == 'video':
            cap = cv2.VideoCapture(input_file)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(os.path.join(output_path, os.path.basename(input_file)), fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))
            while cap.isOpened():
                ret, image = cap.read()
                if not ret:
                    break
                # Detect objects and draw the bounding boxes, class names, and distances
                objects = detect_objects(image, model, scaler)
                indices = cv2.dnn.NMSBoxes([obj['bounding_box'] for obj in objects], [obj['confidence'] for obj in objects], CONF_THRESHOLD, NMS_THRESHOLD)
                for index in indices.flatten():
                    obj = objects[index]
                    xmin, ymin, xmax, ymax = obj['bounding_box']
                    class_name = classes[obj['class_id']]
                    distance = obj['zloc']
                    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                    cv2.putText(image, f'{class_name} {distance:.2f}m', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                out.write(image)
            cap.release()
            out.release()

    cv2.destroyAllWindows()



In [8]:
def process_webcam():
    # Load the class names
    with open('yolov3.names', 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    # Define the confidence and NMS thresholds
    CONF_THRESHOLD = 0.6  # Increase this value to reduce false positives
    NMS_THRESHOLD = 0.5   # Increase this value to allow more overlap between bounding boxes

    # Open the webcam
    cap = cv2.VideoCapture(-1)

    while True:
        # Capture a frame from the webcam
        ret, frame = cap.read()

        # Detect objects and draw the bounding boxes, class names, and distances
        objects = detect_objects(frame, model, scaler)
        indices = cv2.dnn.NMSBoxes([obj['bounding_box'] for obj in objects], [obj['confidence'] for obj in objects], CONF_THRESHOLD, NMS_THRESHOLD)
        #indices = cv2.dnn.NMSBoxes([obj['bounding_box'] for obj in objects], [obj['confidence'] for obj in objects])

        for index in indices.flatten():
            obj = objects[index]
            xmin, ymin, xmax, ymax = obj['bounding_box']
            class_name = classes[obj['class_id']]
            distance = obj['zloc']

            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_name} {distance:.1f}m', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Webcam', frame)

        # Exit the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()


In [9]:

mode = 'image'  # Change this to 'image' or 'video' to process images or videos from a folder

if mode == 'webcam':
    process_webcam()
else:
    input_path = 'testt'
    output_path = 'results_img'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    process_files(input_path, output_path, mode)


/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 17ms/step


/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 18ms/step


/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 17ms/step


/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/sai/.local/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 22ms/step


error: OpenCV(4.7.0) /io/opencv/modules/highgui/src/window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
